In [0]:
%pip install azure-storage-file-datalake

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.4/258.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.6/408.6 kB 41.1 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import pyodbc
import pandas as pd
import requests

# Define the base URL for Ergast API
base_url = "https://ergast.com/api/f1"

In [0]:
jdbcUrl = "jdbc:sqlserver://az-tfo-serv-001.database.windows.net:1433;database=az-tfo-dbs-001"
connectionProperties = {
  "user" : "tfo@az-tfo-serv-001.database.windows.net",
  "password" : "Outlook@123",
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

df = spark.read.jdbc(url=jdbcUrl, table="integration.filemaster", properties=connectionProperties)
#df.createOrReplaceTempView("sql_table")


In [0]:
query = "SELECT SourceUrl,[Table] FROM integration.filemaster"
df = spark.read.jdbc(url=jdbcUrl, table="( " + query + " ) as filemaster", properties=connectionProperties)

# Show the retrieved data
df.show()

+---------+----------+
|SourceUrl|     Table|
+---------+----------+
| circuits|EG_circuit|
|    races|   EG_Race|
+---------+----------+



In [0]:
import requests
import pandas as pd

# Define the base URL for the Ergast API
base_url = "https://ergast.com/api/f1"

# Collect the distinct SourceUrl values from SQL (this could be a list of values)
source_urls = [row['SourceUrl'] for row in df.collect()]
table_names = [row['Table'] for row in df.collect()]

# Prepare a list to store all the DataFrames (one for each SourceUrl)
all_data = []

# Loop through each SourceUrl and call the API
for source_url in source_urls:
    # Build the complete endpoint dynamically
    year = "2023"
    endpoint = f"{base_url}/{year}/{source_url}.json"
               
    
    # Send a GET request to fetch the data
    response = requests.get(endpoint)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        
        # Example for handling 'circuits' endpoint dynamically
        # You need to dynamically normalize data depending on the endpoint
        # Check if the endpoint corresponds to 'circuits' or another table
        if source_url == "circuits":
            data_normalized_circuits = pd.json_normalize(data['MRData']['CircuitTable']['Circuits'])
        elif source_url == "drivers":
            data_normalized_drivers = pd.json_normalize(data['MRData']['DriverTable']['Drivers'])
        elif source_url == "races":
            data_normalized_races = pd.json_normalize(data['MRData']['RaceTable']['Races'])    
        else:
            # Handle other tables if necessary
            data_normalized_items = pd.json_normalize(data['MRData']['OtherTable']['Items'])
        
        # Append the normalized data to the list
        #all_data.append(data_normalized)
        
        # Optional: Print or process the data
        #print(f"Data for {source_url}:")
        #print(data_normalized.head())
    else:
        print(f"Failed to retrieve data from {endpoint}: {response.status_code}")

# After processing all SourceUrls, you can combine all data into a single DataFrame
#final_df = pd.concat(all_data, ignore_index=True)

# Show the final combined data
#print(final_df.head())


In [0]:
# Define catalog, schema (database), and table names
catalog = "projects"
schema = "default"
#table_names = ['circuits', 'races']  # Assuming you have specific table names

# Ensure data_normalized_circuits and data_normalized_races are Spark DataFrames
data_normalized_circuits_spark = spark.createDataFrame(data_normalized_circuits)
data_normalized_races_spark = spark.createDataFrame(data_normalized_races)

# Iterate through each table name and load the corresponding data
for tableName in table_names:
    if tableName == "EG_circuit":
        # Delete the existing table if it exists
        spark.sql(f"DROP TABLE IF EXISTS {catalog}.{schema}.{tableName}")
        # Create and store the table with the circuits data
        data_normalized_circuits_spark.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{schema}.{tableName}")
    elif tableName == "EG_Race":
        # Delete the existing table if it exists
        spark.sql(f"DROP TABLE IF EXISTS {catalog}.{schema}.{tableName}")
        # Create and store the table with the races data
        data_normalized_races_spark.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{schema}.{tableName}")
